## Prepare training data
Training data will be a pandas dataframe grouped by dates. Each date will correspond to all the words of all the articles that day. 
1. TODO: How do we handle weekends? (May cause problems in continuos data)
2. TODO: Drop the days that do not have any articles.

In [64]:
import pickle
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../')
import stock_data

In [2]:
with open("nyt_1800.pkl", "rb") as fp:   # Unpickling
    raw = pickle.load(fp)

In [3]:
df = pd.DataFrame(raw, columns = ["link", "time", "words"])

In [17]:
#Convert to date time type. Sort by date
df['time'] = pd.to_datetime(df.time)
df = df.sort_values('time')
df = df.set_index('time')

In [20]:
df.head()

,link,words
time,,
2018-09-27 00:31:19,https://www.nytimes.com/2018/09/26/sports/manc...,"[MANCHESTER, England, It, was, past, midnight,..."
2018-09-27 04:03:26,https://www.nytimes.com/2018/09/27/briefing/eu...,"[Want, to, get, this, briefing, by, email, Her..."
2018-09-27 04:41:51,https://www.nytimes.com/2018/09/27/world/austr...,"[The, Australia, Letter, is, a, weekly, newsle..."
2018-09-27 09:00:05,https://www.nytimes.com/2018/09/27/travel/how-...,"[When, looking, for, restaurant, recommendatio..."
2018-09-27 09:44:58,https://www.nytimes.com/2018/09/27/briefing/br...,"[Want, to, get, this, briefing, by, email, Her..."


In [37]:
#Convert to String. Handle None type
df['liststring'] = [','.join(map(str, l)) if l is not None else "" for l in df['words'] ]
df.drop(columns= ['words'], inplace = True)

In [46]:
#Concat all the strings of a single day together
post_df = df.groupby(pd.Grouper(freq='D'))['liststring'].apply(lambda x: x.sum())

In [57]:
#Range of dates
print(post_df.head(1).index)
print(post_df.tail(1).index)

DatetimeIndex(['2018-09-27'], dtype='datetime64[ns]', name='time', freq='D')
DatetimeIndex(['2019-05-17'], dtype='datetime64[ns]', name='time', freq='D')


## Prepare labels (which is the stock data) 

In [70]:
#Get the stock data to use as our labels 
label = stock_data.get_stock_data('GOOG', '94Z49Z19XNL1GGGP')

In [72]:
#Range of dates
print(label.head(1).index)
print(label.tail(1).index)

DatetimeIndex(['2014-03-27'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2019-05-20'], dtype='datetime64[ns]', name='date', freq=None)


In [77]:
#Truncate the date range to fit the post_df dataframe
label = label.truncate(before=pd.Timestamp('2018-09-27'), after=pd.Timestamp('2019-05-17'))

In [78]:
label.head(3)

,ds,1. open,2. high,3. low,4. close,5. volume
date,,,,,,
2018-09-27,2018-09-27,1186.73,1202.10,1183.63,1194.64,1260754
2018-09-28,2018-09-28,1191.87,1195.41,1184.50,1193.47,1380629
2018-10-01,2018-10-01,1199.89,1209.90,1190.30,1195.31,1357604


This data doesn't have weekends. Need to think of some kind of way to handle this.